# COMP 472 - Assignment 3

### Bernard Claveau - 40065756 / Nicolas Eliopoulos - 40059378

<hr>

### Imports

In [73]:
import pandas as pd  # ?
import csv  # ?
import math
# tsv_file = open("covid_training.tsv")
# read_tsv = pd.read_csv(tsv_file, delimiter="\t", encoding="utf-8")
# print(read_tsv.head())
# for line in read_tsv:
#     print(line)
file = open("covid_training.tsv", "r", encoding='utf-8')
file = pd.read_csv(file, sep='\t', encoding='utf-8')


def load_dataset(path: str, with_headers = True) -> any:
    file = open(path, "r", encoding='utf-8')

    if with_headers:
        return pd.read_csv(file, sep='\t', encoding='utf-8')
    else:
        return pd.read_csv(file, sep='\t', encoding='utf-8', header = None)
file.head()

,tweet_id,text,q1_label,q2_label,q3_label,q4_label,q5_label,q6_label,q7_label
0,1241025578527903750,For the average American the best way to tell ...,no,NaN,NaN,NaN,NaN,no_joke_or_sarcasm,no_not_interesting
1,1240467080954228737,this is fucking bullshit,no,NaN,NaN,NaN,NaN,no_not_harmful,no_not_interesting
2,1240716889162018816,Can y’all please just follow the government’s ...,no,NaN,NaN,NaN,NaN,no_not_harmful,no_not_interesting
3,1241062514886090754,No offense but the corona virus disappearing b...,no,NaN,NaN,NaN,NaN,no_not_harmful,no_not_interesting
4,1241447017945223169,This is the face of someone who just spent 9 h...,yes,2_no_probably_contains_no_false_info,4_yes_probably_of_interest,1_no_definitely_not_harmful,yes_not_urgent,no_not_harmful,no_not_interesting


## 1. Convert all tweets to lowercase

#### Extract the tweets from the training set

End up with a list of strings where each string is a tweet.

Convert all these strings to lowerase.

In [44]:
file = open("covid_training.tsv", "r", encoding='utf-8')
df = pd.read_csv(file, sep='\t', encoding='utf-8')

df['text'] = df['text'].str.lower()  #Convert to lowercase

## 2. Build a list of all the words in the set

- Separate words by spaces.
- Use words as features and their frequency as the value.

Create a dictionary with words as the keys and their frequency as the value. 

In [47]:
words = {}

for tweet in df.text:
  wit = tweet.split()
  for w in wit:
    if w in words:
      words[w] += 1
    else:
      words[w] = 1


## 3. Create the vocabularies

- Original Vocabulary (NB-BOW-OV): every word in the training set is part of the vocabulary
- Filtered Vocabulary (NB-BOW-FV): only words that appear at least 2 times are part of the vocabulary

In [49]:
ov = words

fv = {}

for w in ov:
  if ov[w] > 1:
    fv[w] = ov[w]
    
print(fv)

{'for': 124, 'the': 453, 'american': 11, 'best': 6, 'way': 11, 'to': 335, 'tell': 5, 'if': 42, 'you': 98, 'have': 65, 'covid-19': 93, 'is': 200, 'cough': 3, 'in': 181, 'a': 262, 'face': 7, 'and': 227, 'their': 27, 'test': 15, 'results': 2, 'this': 121, 'fucking': 6, 'can': 38, 'y’all': 12, 'please': 25, 'just': 38, 'follow': 8, 'so': 23, 'we': 67, 'out': 22, 'be': 76, 'i': 86, 'feel': 6, 'like': 27, 'that': 112, 'losing': 3, 'more': 17, 'time': 13, 'because': 20, 'one': 19, 'or': 34, 'two': 9, 'kids': 3, 'can’t': 4, 'no': 30, 'but': 40, 'corona': 75, 'virus': 65, 'before': 5, 'april': 2, 'actually': 4, 'sucks': 2, 'of': 229, 'someone': 7, 'who': 39, 'spent': 3, '9': 6, 'personal': 4, 'protective': 2, 'ill': 2, 'covid19': 2, 'patients': 10, 'around': 9, '-': 40, 'are': 74, 'only': 17, 'at': 53, 'start.': 2, 'am': 7, 'people,': 6, 'do': 26, 'social': 8, 'distancing': 2, 'self': 7, 'isolation': 2, '#covid19': 47, 'on': 96, 'my': 41, 'door': 4, 'told': 9, 'him': 8, 'he': 36, 'stay': 8, 'fr

## 4. Specs

- **Smoothing:** Use δ = 0.01 smoothing 
    - this means we must also add **0.01 x size of vocabulary** to the denominator
    
- **Log:** Use log<sub>10</sub>

## 5. Train the classifier

v = vocabulary

__

T<sub>total</sub> = total number of tweets

T<sub>yes</sub> = number of tweets labeled **yes**

T<sub>no</sub> = number of tweets labeled **no**

__

TW<sub>yes</sub> = number of words in tweets labeled **yes**

TW<sub>no</sub> = number of words in tweets labeled **no**

w<sub>i-yes</sub> = number of times the word w<sub>i</sub> appears in all tweets labeled **yes**

w<sub>i-no</sub> = number of times the word w<sub>i</sub> appears in all tweets labeled **no**

### P(yes)

Calculate P(yes) using the training set.

P(yes) = T<sub>yes</sub> / T<sub>total</sub>

In [57]:
num_of_yes = df.q1_label.value_counts()['yes']

p_yes = num_of_yes / len(df.q1_label)

0.6190476190476191

### P(no)

Calculate P(no) using the training set.

P(no) = T<sub>no</sub> / T<sub>total</sub>

In [60]:
p_no = 1 - p_yes

### Conditional Probabilities

For **every** word w<sub>i</sub> in the vocabulary, calculate:

(without smoothing)

P'(w<sub>i</sub> | yes) = w<sub>i-yes</sub> / TW<sub>yes</sub>

P'(w<sub>i</sub> | no) = w<sub>i-no</sub> / TW<sub>no</sub>

(with smoothing)

P(w<sub>i</sub> | yes) = (w<sub>i-yes</sub> + 0.01) / (TW<sub>yes</sub> + 0.01 * |v|)

P(w<sub>i</sub> | no) = (w<sub>i-no</sub> + 0.01) / (TW<sub>no</sub> + 0.01 * |v|)

**Note: No need to calulate without smoothing! Just showing the difference.**

In [72]:
# Calculate tw_yes
yes_tweets = df.loc[df['q1_label'] == 'yes'].text
yes_dictionnary = {}
tw_yes = 0
for tweet in yes_tweets:
  wit = tweet.split()
  for w in wit:
    tw_yes += 1
    if w in yes_dictionnary:
      yes_dictionnary[w] += 1
    else:
      yes_dictionnary[w] = 1

no_tweets = df.loc[df['q1_label'] == 'no'].text
no_dictionnary = {}
tw_no = 0
for tweet in no_tweets:
  wit = tweet.split()
  for w in wit:
    tw_no += 1
    if w in no_dictionnary:
      no_dictionnary[w] += 1
    else:
      no_dictionnary[w] = 1

yes_prob = {}
no_prob = {}
for word in ov:
    if word in yes_dictionnary:
        yes_prob[word] = (yes_dictionnary[word] + 0.01) / (tw_yes + 0.01*len(ov))
    else:
        no_prob[word] = (no_dictionnary[word] + 0.01) / (tw_no + 0.01*len(ov))


print(no_prob)

{'average': 0.000266910497774865, 'rich': 0.000266910497774865, 'person’s': 0.000266910497774865, 'wait': 0.000266910497774865, 'bullshit': 0.000266910497774865, 'government’s': 0.000266910497774865, 'instructions': 0.000266910497774865, 'knock': 0.000266910497774865, 'done?!': 0.000266910497774865, 'kindergartner': 0.000266910497774865, 'keeps': 0.000266910497774865, 'losing': 0.0007954461369330133, 'recess': 0.000266910497774865, 'directions.': 0.000266910497774865, 'offense': 0.000266910497774865, 'disappearing': 0.000266910497774865, 'sucks': 0.0005311783173539391, 'over,': 0.0007954461369330133, 'trash': 0.000266910497774865, '“low': 0.000266910497774865, 'end”': 0.000266910497774865, 'grocery': 0.000266910497774865, 'store,': 0.000266910497774865, 'dollar': 0.000266910497774865, 'fast': 0.000266910497774865, 'food': 0.0010597139565120875, 'walmart': 0.000266910497774865, 'employees,': 0.000266910497774865, 'deserved': 0.000266910497774865, 'wage': 0.000266910497774865, 'survive':

In [ ]:
# For every word in yes, Calculate worth

## 6. Classify a new tweet

For both classes, **yes** and **no**, calculate the score:

score(yes) = log(P(yes)) + ∑ log(P(w<sub>i</sub> | yes))

score(no) = log(P(no)) + ∑ log(P(w<sub>i</sub> | no))

and classify the tweet according to the higher score.

In [104]:
def score_yes(tweet):
    tweet = tweet.lower()
    words = tweet.split()
    score = math.log10(p_yes)
    for word in words:
        if word in yes_prob:
            score += math.log10(yes_prob[word])
    return score

def score_no(tweet):
    tweet = tweet.lower()
    words = tweet.split()
    score = math.log10(p_no)
    for word in words:
        if word in no_prob:
            score += math.log10(no_prob[word])
    return score

def classify(tweet):
    yes, no = score_yes(tweet), score_no(tweet)
    return (yes, "Yes") if yes > no else (no, "No")
    
score, classification = classify("@realdonaldtrump hamburgers")
print("{} {}".format(score, classification))

-3.077490069928598 Yes


## 7. Calculate performance metrics

- accuracy (acc)
- per-class precision (yes-P, no-P)
- per-class recall (yes-R, no-R)
- per-class F1-measure (yes-F, no-F)